# Answering Business Questions for Chinook Record Store Using SQL

## 1. Introduction

In this project we are going to answer business questions related to *Chinook Record Store* using SQL. Chinook database is provided as a SQLite database file. Down below we impliment the code which connects the Jupyter Notebook to the Chinook databse.

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

Further we will run few queries in order to familiarize ourselves with the data.First we will look at the list of all tables in the database.

In [2]:
%%sql
SELECT name, type
  FROM sqlite_master
 WHERE type IN ("table", "view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


As we can see from the above list there are total of *eleven* tables in our database. We will look at couple of tables from the above list and get familiar with the data.

In [3]:
%%sql
SELECT *
  FROM album
 LIMIT 5;   

Done.


album_id,title,artist_id
1,For Those About To Rock We Salute You,1
2,Balls to the Wall,2
3,Restless and Wild,2
4,Let There Be Rock,1
5,Big Ones,3


In [4]:
%%sql
SELECT *
  FROM playlist
 LIMIT 5;   

Done.


playlist_id,name
1,Music
2,Movies
3,TV Shows
4,Audiobooks
5,90’s Music


## 2.Finding Which Genres Sell Best in USA

The Chinook record store has signed a deal with a new record label. We have to select first three albums from a list of four, to be added to the store. All four albums are by artists who do not possess any tracks in the store. Below are the artist and genre names.

|*Artist Name*	      |       *Genre*  |
|---------------------|----------------|
|Regal	              |        Hip-Hop |
|Red Tone	          |        Punk    | 
|Meteor and the Girls |  	      Pop  | 
|Slim Jim Bites	      |        Blues   | 

The record label specializes in artists from USA. They have given Chinook some money to advertise the new albums in USA. Our objective is to find out which genres sell best in USA.

In [5]:
%%sql
WITH 
Genre_Sold_USA AS 
(
 SELECT 
    g.name genre,
    SUM(il.quantity) Sum_Sold
  FROM genre g
 LEFT JOIN track t ON t.genre_id = g.genre_id   
 LEFT JOIN invoice_line il ON il.track_id = t.track_id
 LEFT JOIN invoice i ON i.invoice_id = il.invoice_id
 LEFT JOIN customer c ON c.customer_id = i.customer_id
 
 WHERE c.country="USA"
 GROUP BY 1
 ORDER BY 2 DESC   
    )

SELECT GSU.genre Genre,
       GSU.Sum_Sold Absolute_Sold,
       ROUND(GSU.Sum_Sold*100 / (SELECT TOTAL(GSU.Sum_Sold) FROM Genre_Sold_USA GSU), 2) Percentage_Sold
  FROM Genre_Sold_USA GSU;  


Done.


Genre,Absolute_Sold,Percentage_Sold
Rock,561,53.38
Alternative & Punk,130,12.37
Metal,124,11.8
R&B/Soul,53,5.04
Blues,36,3.43
Alternative,35,3.33
Latin,22,2.09
Pop,22,2.09
Hip Hop/Rap,20,1.9
Jazz,14,1.33


From the above list it is clear that *Rock* sells best in USA, i.e. *53.38%* (more than half). But we do not have it in the record label. 

**Below we list all the four genres in the record label according to their sales percentage (highest to lowest) in USA.**

 - Highest of these is *Alternate & Punk*. It has a sales of *12.37%*. And *Punk* is in the record label.
 - Next highest is *Blues* with the sales of *3.43%*.
 - Third highest is *Pop* with the sales of *2.09%*.
 - Last is *Hip-Hop/Rap* with only *1.9%* sales. 
 
 
 **Based on our observation of the sales percentage, we recommend these three artists:-**
 
 1.Red Tone
 
 2.Slim Jim Bites
 
 3.Meteor and the Girls

## 3. Analyzing Variance in Employee (Sales Support Agent) Performance

Each customer for the Chinook store gets assigned to a *sales support agent* from the company, when they purchase tracks. Here we will analyze if any sales support agent is performing better/worse than others. We will also consider any column (information) from the *employee* table that is indicative of variance in their performance. 

In [6]:
%%sql
SELECT e.first_name || " " || e.last_name Employee_Name,
       e.title Title,
       e.hire_date Hire_Date,
       e.birthdate Birthdate,
       max(invoice_date) latest_invoice_date,
       ROUND(SUM(i.total), 2) Total_Sales_$,
       ROUND(SUM(i.total) * 100 /(SELECT SUM(total)
                                    FROM invoice), 2) Sales_Perc
       
  FROM employee e

LEFT JOIN customer c ON c.support_rep_id = e.employee_id
LEFT JOIN invoice i ON i.customer_id = c.customer_id

WHERE Title = "Sales Support Agent"
GROUP BY 1
ORDER BY 6 DESC;

Done.


Employee_Name,Title,Hire_Date,Birthdate,latest_invoice_date,Total_Sales_$,Sales_Perc
Jane Peacock,Sales Support Agent,2017-04-01 00:00:00,1973-08-29 00:00:00,2020-12-30 00:00:00,1731.51,36.77
Margaret Park,Sales Support Agent,2017-05-03 00:00:00,1947-09-19 00:00:00,2020-12-29 00:00:00,1584.0,33.63
Steve Johnson,Sales Support Agent,2017-10-17 00:00:00,1965-03-03 00:00:00,2020-12-20 00:00:00,1393.92,29.6


From the above table we see that *Jane Peacock* holds the highest sales amount of *~ \$1731*, followed by *Margaret Park* with *~ \$1584*. Last is *Steve Johnson* with *~ \$1394* sales amount. 

If we look at their *hire_date*, *Jane Peacock* joined first i.e. on *4th April 2017*. *Margaret Park* joined in a month's time i.e. on *3rd May 2017*. Where as *Steve Johnson* joined a few month's later, i.e. on *17th October 2017*. If we look at the latest *invoice date* of all the three, there is not much difference. 

We observe that both *Jane Peacock* and *Steve Johnson* hold better sales value compared to *Margaret Park*, evethough *Jane Peacock* joined a month earlier and *Steve Johnson* joined five months later than her. If we Look at the birthdate, both *Jane Peacock* and *Steve Johnson* are younger to *Margaret Park* and *Jane Peacock* is younger to *Steve Johnson*. From this we perceive that the age can also have an effect on the performance of the employees. i.e. younger employees  perform better than older employees.

## 4. Analyzing the Sales Data for Customers from Each Country

In this section we are going to analyze the sales data for customers from each different country. Here we are going to calculate
 - Total number of customers
 - Total value of sales
 - Average value of sales per customer
 - Average order value
 
For each country.

As there are many countries with only one customer, we will group these customers into **Others** in our analysis and put them last in the table. 

First let us calculate the *total number of customers* for each country.

In [7]:
%%sql
WITH num_of_cstmrs AS
(SELECT country Country,
       count(first_name) No_of_customers       
  FROM customer 
GROUP BY 1
ORDER BY 2 DESC)  

SELECT * FROM num_of_cstmrs;

Done.


Country,No_of_customers
USA,13
Canada,8
Brazil,5
France,5
Germany,4
United Kingdom,3
Czech Republic,2
India,2
Portugal,2
Argentina,1


First we will calculate the *total value of sales*, *average value of sales per customer* and *average order value* for all countries. Then we can seperate the countries with only one customer, aggregate their values and rename them as *Others*. Later we can club the countries with more than one customer and countries with one customer into one table. 

In [8]:
%%sql
WITH tot_sales_val AS
(SELECT c.country Country,
       ROUND(SUM(i.total), 2) Tot_Value_Sales,
       ROUND(AVG(i.total), 2) Avg_Order_Value,
       ROUND(SUM(i.total)/(SELECT COUNT(DISTINCT(customer_id)) FROM invoice), 2) Avg_Sales_Value_per_Customer
 
 FROM customer c
INNER JOIN invoice i ON i.customer_id = c.customer_id
GROUP BY 1
ORDER BY 2 DESC),

sales_data_1 AS
(SELECT noc.Country,
       noc.No_of_customers,
       tsv.Tot_Value_Sales,
       tsv.Avg_Sales_Value_per_Customer,
       tsv.Avg_Order_Value
  FROM Num_of_cstmrs noc  
INNER JOIN tot_sales_val tsv ON tsv.Country=noc.Country
 WHERE noc.No_of_customers > 1),

sales_data_2 AS
(SELECT CASE
       WHEN noc.No_of_customers=1 THEN "Others"
       END AS
       Country,
       noc.No_of_customers,
       ROUND(AVG(tsv.Tot_Value_Sales),2) Tot_Value_Sales,
       ROUND(AVG(tsv.Avg_Sales_Value_per_Customer),2) AVG_Sales_Value_per_Customer,
       ROUND(AVG(tsv.Avg_Order_Value),2) AVG_Order_Value
  FROM Num_of_cstmrs noc  
INNER JOIN tot_sales_val tsv ON tsv.Country=noc.Country
 WHERE noc.No_of_customers = 1),

tot_sales_data AS
(SELECT * 
FROM sales_data_1
UNION
SELECT * 
FROM sales_data_2)

SELECT *
  FROM tot_sales_data
ORDER BY 3 DESC;

Done.


Country,No_of_customers,Tot_Value_Sales,Avg_Sales_Value_per_Customer,Avg_Order_Value
USA,13,1040.49,17.64,7.94
Canada,8,535.59,9.08,7.05
Brazil,5,427.68,7.25,7.01
France,5,389.07,6.59,7.78
Germany,4,334.62,5.67,8.16
Czech Republic,2,273.24,4.63,9.11
United Kingdom,3,245.52,4.16,8.77
Portugal,2,185.13,3.14,6.38
India,2,183.15,3.1,8.72
Others,1,73.0,1.24,7.44


Above table is in the descending order of *total value of sales*. And *Others* (countries with only one customer) is at the bottom of the table.

Here we observe that USA is on top of the list with *13* customers and with ~ \$1040 total value of sales. Next is CANADA with *8* customers and ~ \$535 total value of sales.

## 5. Analyzing the Invoice Data: Album Purchase Vs Individual Purchase of Tracks

Chinook store allows the customers to make purchases in two ways:
* purchase a whole album
* purchase a collection of one or more individual tracks

But the store does not allow the customers to purchase a whole album and then add individual tracks to the same purchase. The cost of whole album is same as that of each track if purchased seperately.

The company wants to change their policy inorder to save money. The new stratergy it wants to consider is to purchase only the popular tracks from each album from the record companies, instead of purchasing every track from an album.

Our task is to calculate the percentage of individual tracks purchased Vs percentage of whole album purchased by the customers. This analysis can help the company understand the impact of new statergy on overall revenue.

We are going to accomplish this task by analyzing the tracks from invoice and then comparing it with the album. 

First let us look at the Number of invoices.

In [9]:
%%sql
SELECT MAX(invoice_id) total_invoice
  FROM invoice_line;

Done.


total_invoice
614


From the above table we observe that there are total of *614* invoices. Further we will compare the tracks from invoice with that of albums.

In [10]:
%%sql
WITH com_pur AS
(SELECT il.invoice_id invoice_id, il.track_id Pur_tracks, t.album_id album_id
  FROM track t
INNER JOIN invoice_line il ON il.track_id = t.track_id
ORDER BY 1)

SELECT *
  FROM com_pur;

Done.


invoice_id,Pur_tracks,album_id
1,1158,91
1,1159,91
1,1160,91
1,1161,91
1,1162,91
1,1163,91
1,1164,91
1,1165,91
1,1166,91
1,1167,91


From the above table we observe that if the tracks belonging to each distinct invoice, belongs to the same album it is an *Album Purchase*. If these tracks belong to different albums it is *Non Album Purchase*. 

Below we categorize these invoices into *Album and Non Album Purchase*. We do this by counting the number of distinct albums belonging to each invoice. Then we compare the *Distinct Albums* and *Number of Albums* belonging to each invoice.

In [11]:
%%sql
WITH com_pur AS
(SELECT il.invoice_id invoice_id, il.track_id Pur_tracks, t.album_id album_id
  FROM track t
INNER JOIN invoice_line il ON il.track_id = t.track_id
ORDER BY 1)

SELECT invoice_id, COUNT(DISTINCT(album_id)) Distinct_album_id, 
       COUNT(album_id) No_of_album_id,
       CASE
       WHEN COUNT(album_id) - COUNT(DISTINCT(album_id)) <= 2 THEN "Non Album Purchase"
       ELSE "Album Purchase"
       END AS Category 
  FROM com_pur
GROUP BY 1;

Done.


invoice_id,Distinct_album_id,No_of_album_id,Category
1,1,16,Album Purchase
2,10,10,Non Album Purchase
3,2,2,Non Album Purchase
4,8,8,Non Album Purchase
5,1,17,Album Purchase
6,2,2,Non Album Purchase
7,11,11,Non Album Purchase
8,10,10,Non Album Purchase
9,9,9,Non Album Purchase
10,2,2,Non Album Purchase


### 5.1 Category: Not an Album Purchase

Here we calculate the *Percentage of Non Album Purchases* for the company.

In [12]:
%%sql
WITH com_pur AS
(SELECT il.invoice_id invoice_id, il.track_id Pur_tracks, t.album_id album_id
  FROM track t
INNER JOIN invoice_line il ON il.track_id = t.track_id
ORDER BY 1),

ctgr_pur AS
(SELECT invoice_id, COUNT(DISTINCT(album_id)) Distinct_album_id, 
       COUNT(album_id) No_of_album_id,
       CASE
       WHEN COUNT(album_id) - COUNT(DISTINCT(album_id)) <= 2 THEN "Non Album Purchase"
       ELSE "Album Purchase"
       END AS Category 
  FROM com_pur
GROUP BY 1)

SELECT (SELECT MAX(invoice_id)
          FROM invoice_line) TOTAL_invoice_id,
       COUNT(invoice_id) No_of_invoice_id,
       (SELECT ROUND(CAST(COUNT(invoice_id) AS FLOAT) * 100 / (SELECT MAX(invoice_id)
                                            FROM invoice_line), 2)) Perc_invoice
  FROM ctgr_pur
WHERE Category = "Non Album Purchase";

Done.


TOTAL_invoice_id,No_of_invoice_id,Perc_invoice
614,480,78.18


From the above table we deduce that out of *614* invoices, *480* invoices are *Non Album Purchases*. i.e. *~ 78%* of purchases are *Non Album Purchases*.  

### 5.2 Category: Album Purchase

Here we calculate the *Percentage of Album Purchase* for the company.

In [13]:
%%sql

WITH com_pur AS
(SELECT il.invoice_id invoice_id, il.track_id Pur_tracks, t.album_id album_id
  FROM track t
INNER JOIN invoice_line il ON il.track_id = t.track_id
ORDER BY 1),

ctgr_pur AS
(SELECT invoice_id, COUNT(DISTINCT(album_id)) Distinct_album_id, 
       COUNT(album_id) No_of_album_id,
       CASE
       WHEN COUNT(album_id) - COUNT(DISTINCT(album_id)) <= 2 THEN "Non Album Purchase"
       ELSE "Album Purchase"
       END AS Category 
  FROM com_pur
GROUP BY 1)


SELECT (SELECT MAX(invoice_id)
  FROM invoice_line) TOTAL_invoice_id,
       COUNT(invoice_id) No_of_invoice_id,
      (SELECT ROUND(CAST(COUNT(invoice_id) AS FLOAT) * 100 / (SELECT MAX(invoice_id)
                                           FROM invoice_line), 2)) Perc_invoice
 FROM ctgr_pur
WHERE Category = "Album Purchase";

Done.


TOTAL_invoice_id,No_of_invoice_id,Perc_invoice
614,134,21.82


From the above table we deduce that out of *614* invoices, only *134* are *Ablum Purchases*. i.e. *~ 22%* of total purchases is an *Album Purchase*.

From the analysis it is clear that the individual track purchases (~ 78%) is comapartively higher than album purchases (~ 22%).  And we infer that the store should consider buying popular tracks from albums rather than buying the whole album from record companies.

## 6. Extra Queries

### 6.1. Artist Used in Most Playlists

In this querry we are going to find out *which artist is used in most playlists*. For this analysis we first join *playlist table* and *artist table* using *INNER JOIN* method. Then we will use *GROUP BY* method on *artists* and count the number of *playlists*. The artist with maximum number of playlists is in most playlists. 

In [14]:
%%sql
WITH pl_ar AS
(SELECT p.name playlists, at.name artists
   FROM playlist p
 INNER JOIN playlist_track pt ON pt.playlist_id = p.playlist_id
 INNER JOIN track t ON t.track_id = pt.track_id
 INNER JOIN album a ON a.album_id = t.album_id
 INNER JOIN artist at ON at.artist_id = a.album_id)

SELECT playlists, artists, COUNT(*) No_of_playlists
  FROM pl_ar
GROUP BY 2
ORDER BY 3 DESC;

Done.


playlists,artists,No_of_playlists
Music,The Police,143
Brazilian Music,Vinícius E Qurteto Em Cy,91
Brazilian Music,Frank Zappa & Captain Beefheart,87
Music,Marcos Valle,69
Music,Barry Wordsworth & BBC Concert Orchestra,66
Music,Big & Rich,63
Music,Page & Plant,60
Music,Ed Motta,59
Music,Berliner Philharmoniker & Herbert Von Karajan,57
Music,Fernanda Porto,57


From the above table we can conclude that **The Police** with *143* numbers is used in most playlists.

### 6.2. Tracks Purchased Vs Not Purchased

In this section we are going to calculate the *number of tracks purchased* Vs *number of tracks not purchased*. First let us calculate how many tracks are there with the *Chinook* store.

In [15]:
%%sql
SELECT MAX(track_id) No_of_tracks
  FROM track;

Done.


No_of_tracks
3503


As we can see there are a total of **3503** number of tracks with the store. Now we can calculate the number of tracks which are purchased Vs not purchased. For this purpose we will use *LEFT JOIN* operator on *track table* (this has all the tracks) and *invoice_line* (this has tracks which are sold). 

In [16]:
%%sql
WITH TRACKS AS
(SELECT t.track_id all_tracks, COUNT(il.track_id) purchased_tracks,
        CASE
        WHEN COUNT(il.track_id) == 0 THEN "Not Purchased"
        ELSE "Purchased"
        END AS Category 
   FROM track t
 LEFT JOIN invoice_line il ON il.track_id = t.track_id
 GROUP BY 1)

SELECT COUNT(Category)
  FROM TRACKS
 WHERE Category = "Purchased";    

Done.


COUNT(Category)
1806


In [17]:
%%sql
WITH TRACKS AS
(SELECT t.track_id all_tracks, COUNT(il.track_id) purchased_tracks,
        CASE
        WHEN COUNT(il.track_id) == 0 THEN "Not Purchased"
        ELSE "Purchased"
        END AS Category 
   FROM track t
 LEFT JOIN invoice_line il ON il.track_id = t.track_id
 GROUP BY 1)

SELECT COUNT(Category)
  FROM TRACKS
 WHERE Category = "Not Purchased";     

Done.


COUNT(Category)
1697


From the above analysis we observe the following points:-

 * Total number of tracks purchased is **1806**
 * Number of tracks which are not purchased is **1697**
 
 We can confirm that this analysis is correct by adding the above two numbers, as this amounts to total number of tracks the store owns i.e. 3503. 

### 6.3. Reflection of Range of Tracks on Sales Popularity

In this section we are going to see wether the *range of tracks* with *Chinook* store reflects its sales popularity. For this analysis we will compare the number of tracks the company has in each genre with it's sales value.

In [18]:
%%sql
SELECT g.name genre_name, COUNT(t.track_id) no_of_tracks, COUNT(il.track_id) quantity_sold, ROUND(CAST(COUNT(il.track_id) AS FLOAT) * 100.0 / CAST(COUNT(t.track_id) AS FLOAT), 2) perc_sales
  FROM genre g
INNER JOIN track t ON t.genre_id = g.genre_id    
LEFT JOIN invoice_line il ON il.track_id = t.track_id    
GROUP BY 1
ORDER BY 2 DESC;   

Done.


genre_name,no_of_tracks,quantity_sold,perc_sales
Rock,3017,2635,87.34
Metal,755,619,81.99
Alternative & Punk,648,492,75.93
Latin,627,167,26.63
Jazz,190,121,63.68
R&B/Soul,165,159,96.36
Blues,149,124,83.22
Alternative,123,117,95.12
Classical,105,47,44.76
TV Shows,93,2,2.15


As we can see the table above, there are total of *18* different genres. 
 - *Rock* tops the list with highest number of tracks i.e. *2635*. It has a good sales percentage is i.e.*~ 87%*. Next is *Metal* with *755* total number of tracks and *~ 82%* sales percentage, we can consider this also as a good sales value. Third is *Alternative & Punk* with *648* total number of tracks with only *~ 76%* of sales percentage, which is a moderate sales value. 
 - There are some genres with large number of tracks but low sales percentage. For e.g. *Latin* has *627* number of tracks and has only *~ 26%* sales value. *Classical*, *Heavy Metal*, *TV Shows*, *Reggae* have below *50%* sales percentage.
 - There are some other genres where number of tracks are less but higher sales value. Examples are *Easy Listenng*, *R&B/Soul*, *Alternative*, *Blues* etc. 
 
Overall, we observe that the number of tracks does reflect the sales popularity with slight discrepancies. 

### 6.4. Effect of Protected and Non-protected Media Types on Popularity

In this section we will find out if *protected* and *non-protected media types* have any effect on the popularity. Here we have five different media types.

 1. **Protected Media Types**
    1. Protected AAC audio file
    2. Protected MPEG-4 video file     
    
 2. **Non- Protected Media Types**
    1. Purchased AAC audio file
    2. MPEG audio file
    2. AAC audio file    

In [19]:
%%sql
SELECT mt.name media_type, COUNT(*) total_tracks, COUNT(il.quantity) track_sold, ROUND(CAST(COUNT(il.quantity) AS FLOAT) * 100.0/CAST(COUNT(t.track_id) AS FLOAT), 2) perc_sales 
  FROM media_type mt
INNER JOIN track t ON t.media_type_id = mt.media_type_id
LEFT JOIN invoice_line il ON il.track_id = t.track_id
GROUP BY 1
ORDER BY 2 DESC;

Done.


media_type,total_tracks,track_sold,perc_sales
MPEG audio file,5652,4259,75.35
Protected AAC audio file,525,439,83.62
Protected MPEG-4 video file,214,3,1.4
Purchased AAC audio file,39,35,89.74
AAC audio file,24,21,87.5


Above table lists all the media types and thier sales percentage. 

 - Highest number of tracks the comany holds is *MPEG audio file* with *5652* number of tracks. Out of this *4259* tracks are sold i.e. *~ 75%*. Among other non-protected media types (*Purchased AAC audio file* and *AAC audio file*), the company holds fewer number of tracks and the sales percentage is above *85%*. 
 - The company holds *525 Protected AAC audio file* and it has *~ 83%* sales percentage. This is a good sales value. Company also holds *214 Protected MPEG-4 audio file* among this only *1.4%* of tracks are sold, this is way too low. 
 
We observe that among all the media-types *Protected MPEG-4 video file* is not popular among the consumers. Even though *MPEG audio file* sales percentage is good, not as high as other audio files. Otherwise, we do not see any effect on popularity between protected and non protected media types.     

## Conclusions

In this project we answered business queries related to *Chinook Record Store* using *SQL* Database. Our analysis led to the following observations:-

 * We found which genres sell best in the *USA*. Based on this we listed three albums to be added to the *Chinook store*. These are:-
   * Red Tone
   * Slim Jim Bites
   * Meteor and the Girls
 * We analyzed the performance of *Sales Support Agents*. And found that *Jane Peacock* and *Steve Johnson* are doing better than *Margaret Parker*. We also observed that the age impacts the performance as *Jane Peacock* is youngest of all and *Steve Johnson* is younger to *Margaret Parker*. 
 * We analyzed the sales data for customers from each country. We calculated *total number of customers*, *total value of sales*, *average value of sales per customer* and *average order value* for each country. We observed that *USA* is on top of the list with *13* customers and *~ \$1040* total value of sales. Next is *CANADA* with *8* customers and with *~ \$535* value of sales.
 * We calculated the percentage of individual tracks and  the percentage of whole album purchased by customers. Our analysis led to the conclusion that the individual tracks purchases (~ 78%) are higher compared to album purchases (~ 22%). This can help the company in deciding whether to buy popular tracks from albums or to buy the whole album from record companies.
 * We analyzed which artist is used in most playlists and found that *The Police* is used in most playlists.
 * We calculated the number of tracks purchased Vs not purchased. Of all *3503* tracks, we found *1806* tracks purchased and *1697* tracks not purchased.
 * We analyzed the sales popularity of range of tracks the *Chinook Store* owns. For this we calculated the number of tracks company owns in each genre and it's sales value. We observed that the number of tracks does reflect the sales popularity with slight discrepancies. 
 * We observed if different media types have any effect on popularity. We observe that among all the media-types *Protected MPEG-4 video file* is not popular among the consumers. Also *MPEG audio file* sales percentage is not as high as *Protected AAC audio files*, *Purchased AAC audio files* and *AAC audio files*. Otherwise, we do not see any effect on popularity between protected and non protected media types.